In [2]:
import pandas as pd
import numpy as np

In [9]:
#Creamos la función
def concatenar(start,end,verbose=False):    
    #Decimos la ruta de los archivos
    #basepath = "/media/jonathan/JHON/Tesis/DatosTesis/raw_data/%.4d%.2d%.2d_maestra_1_mitma_distrito.csv"
    basepath = "/media/jonathan/JHON/Tesis/DatosTesis/MatrizDeViajes/Distritos/FicherosDiarios/1erPeriodo/%.4d%.2d%.2d_maestra_1_mitma_distrito.csv"
    #Definimos una lista con las fehas de inicio y final que se analizarán, esta lista se guardará en "dates"
    dates = pd.date_range(start, end)
    
    #El simbolo % es el operador de modulo(residuo de un problema de división) 
    #El simbolo % en cadenas compara si dos textos son iguales y arroja un True or False(Carro=Caro->False)
    #Se hace un ciclo for para capturar cada timestamp y luego se pasa cada una de las fechas al basepath
    paths = [basepath % (timestamp.year, timestamp.month, timestamp.day) for timestamp in dates]

    n = 0
    
    #Creamos 2 dataframe vacios
    timeseries_o = pd.DataFrame({})
    timeseries_d = pd.DataFrame({})
    
    #Hacemos un recorrido por cada uno de los paths (nombre completo del archivo) 
    for path in paths:
        
        #Verifica si la variable de función "verbose esta en True", Si es asi entonces imprime el porcentaje de archivos concatenados
        if verbose: print(path, "%.2f%%" % (float(n) * 100 / len(paths)))
        
        #Leemos cada uno de los archivos csv y lo guardamos en la variable "data"
        #Le datos a las columnas de fEcha, origen y destino un formato tipo string
        
        data = pd.read_csv(path, dtype={"fecha": str,
                                        "origen": str,
                                        "destino": str}, sep='|', usecols=['fecha','origen','viajes','distancia'],index_col=0, parse_dates=True)
        
        data2 = pd.read_csv(path, dtype={"fecha": str,
                                        "origen": str,
                                        "destino": str}, sep='|', usecols=['fecha','destino','viajes','distancia'],index_col=0, parse_dates=True)
        
        timeseries_o = timeseries_o.append(data)
        timeseries_d = timeseries_d.append(data2)
        print(timeseries_o)
        
        #Acotamos los datos y solo utlizamos aquellos datos que tienen una distancia mayor a 002-005        
        #data = data[data.distancia >= "010-050"]
        #print(data)
        
        #data = data[data.distancia >= "002-005"]
        
        #El método pivot_table permite reorganizar en forma de tabla un dataframe,las columnas de esta tabla
        #ahora son los distritos de "origen", el periodo (hora del dia) ahora es el indice y representa las filas 
        #de la tabla y los datos que se presentan corresponden al # de viajes en determinada hora del día (00-23) 
        #dependiendo del distrito de origen.  
        #table_o = data.pivot_table("viajes", index="periodo", columns=["origen"], aggfunc=np.sum)        
        
        '''
        table_o = data.pivot_table(data, values='origen','viajes','distancia', columns=['origen','viajes','distancia'], index="periodo")
        print(table_o)
        
        #Cambiamos a tipo flotante los valores de la tabla creada
        table_o = table_o.astype(np.float32)
        
        #Consulta la forma del indice, en este caso el indice es la variable periodo (00-23) tiene que dar 24,
        #por las 24 horas del día. 
        #Se hace para proteger de una excepción si el distrito en cuestion no tiene registros de viajes 
        
        if table_o.index.shape[0] != 24:  
            j = 0                           
            for i in range(24):
                if table_o.index[i] != (i+j):
                    print (data.fecha.iloc[0],i+j,"origen sin registros!")
                    table_o.loc[i+j] = 0
                    j += 1
                if (i + j) == 23: break
            table_o = table_o.sort_index() #sort_index devuelve un nuevo datafRame ordenado por etiqueta si la table es False, de lo contrario actualiza la tabla y retorna un NONE
            
        #Crea una nueva columa "ds" con la fecha, fecha.iloc[0] devuelve "20200214"-"20200215"... y la hora en formato 24h        
        table_o["ds"] = pd.date_range(data.fecha.iloc[0], "%s 23:00:00" % (data.fecha.iloc[0]), freq="H")
        
        #Fija como indice la columna ds
        table_o = table_o.set_index("ds")
        
        #los datos que no tengan registros y aparezcan como NaN el metodo fillna los pondrá en 0
        table_o = table_o.fillna(0)
        
        #Añade los datos de los diferentes archivos a una sola variable llamada timeseries_o
        timeseries_o = timeseries_o.append(table_o)
        

        table_d = data.pivot_table("viajes", "periodo", ["destino"], aggfunc='sum')
        table_d = table_d.astype(np.float32)
        if table_d.index.shape[0] != 24:
            j = 0
            for i in range(24):
                if table_d.index[i] != (i + j):
                    print(data.fecha.iloc[0], i + j, "destino sin registros!")
                    table_d.loc[i + j] = 0
                    j += 1
                if (i + j) == 23: break
            table_d = table_d.sort_index()
        table_d["ds"] = pd.date_range(data.fecha.iloc[0], "%s 23:00:00" % (data.fecha.iloc[0]), freq="H")
        table_d = table_d.set_index("ds")
        table_d = table_d.fillna(0)
        timeseries_d = timeseries_d.append(table_d)

        n += 1
        
        '''
        
    return timeseries_o, timeseries_d

In [10]:
start = "20200214"
end = "20200215"
timeseries_o,timeseries_d = concatenar(start,end)

#timeseries_o.to_csv("2dointento_o.csv")
#timeseries_d.to_csv("2dointento_d.csv")

/tmp/ipykernel_7146/1256356991.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeseries_o = timeseries_o.append(data)
/tmp/ipykernel_7146/1256356991.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeseries_d = timeseries_d.append(data2)


              origen distancia  viajes
fecha                                 
2020-02-14  01001_AM   002-005   9.775
2020-02-14  01001_AM   005-010   8.754
2020-02-14  01001_AM   002-005   8.431
2020-02-14  01001_AM   010-050   9.775
2020-02-14  01001_AM   002-005   6.007
...              ...       ...     ...
2020-02-14   5200108  0005-002  62.860
2020-02-14   5200108  0005-002  23.569
2020-02-14   5200108   002-005  11.225
2020-02-14   5200108  0005-002  11.225
2020-02-14   5200108   002-005  11.225

[7232427 rows x 3 columns]


/tmp/ipykernel_7146/1256356991.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeseries_o = timeseries_o.append(data)
/tmp/ipykernel_7146/1256356991.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeseries_d = timeseries_d.append(data2)


              origen distancia  viajes
fecha                                 
2020-02-14  01001_AM   002-005   9.775
2020-02-14  01001_AM   005-010   8.754
2020-02-14  01001_AM   002-005   8.431
2020-02-14  01001_AM   010-050   9.775
2020-02-14  01001_AM   002-005   6.007
...              ...       ...     ...
2020-02-15   5200108   002-005  10.577
2020-02-15   5200108  0005-002  40.256
2020-02-15   5200108  0005-002  34.613
2020-02-15   5200108  0005-002  12.700
2020-02-15   5200108   002-005  10.023

[13578411 rows x 3 columns]
